In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.PowerShell,*-*"
#r "nuget:Microsoft.DotNet.Interactive.Http,*-*"
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"
#r "nuget:FSharp.Core, *-*"
#r "nuget:Plotly.NET,*-*"
#r "nuget: Plotly.NET.Interactive, *-*"
using Plotly.NET;

In [ ]:
#!connect mssql --kernel-name splSnapshots "Server=localhost;Database=SeattlePublicLibraryOpenData;Trusted_Connection=True;TrustServerCertificate=true;"

Kernel added: #!sql-splSnapshots

In [ ]:
#!sql-splSnapshots --name itemTypeAgeGroup


;WITH ageGroup_cte
AS (SELECT
  ReportDate,
  AgeGroup,
  COUNT(DISTINCT bibnum) AS ItemCount,
  SUM(ei.itemcount) AS InventoryCount

FROM ElementInventory ei
INNER JOIN ItemType it
  ON ei.ItemTypeCode = it.Code
WHERE ei.ItemCount > 0
GROUP BY ReportDate,
         AgeGroup),
calculatedReportDateTotal_cte
AS (SELECT
  ReportDate,
  SUM(itemcount) AS itemCountSum,
  SUM(inventoryCount) AS inventoryCountSum
FROM ageGroup_cte
GROUP BY ReportDate)
SELECT
  r.ReportDate,
  AgeGroup,
  CAST(ItemCount AS float) / itemCountSum * 100 AS AgeGroupItemPercentage,
  CAST(InventoryCount AS float) / inventoryCountSum * 100 AS AgeGroupInventoryPercentage

FROM calculatedReportDateTotal_cte r
INNER JOIN ageGroup_cte a
  ON a.ReportDate = r.ReportDate

ORDER BY ReportDate, AgeGroup




In [ ]:
#!share --from sql-splSnapshots itemTypeAgeGroup

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;





#!html
<a id="ageGroup"></a>

In [ ]:
#!sql-splSnapshots --name itemTypeFormat

SELECT ReportDate,
       FormatGroup,
       Count(DISTINCT bibnum) as ItemCount,
       Sum(ei.itemcount) as InventoryCount
FROM   ElementInventory ei
       INNER JOIN ItemType it
               ON ei.ItemTypeCode = it.Code

GROUP  BY ReportDate,
          FormatGroup 

In [ ]:
#!share --from sql-splSnapshots itemTypeFormat

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;



In [ ]:
#!sql-splSnapshots --name itemCollectionFormat

-- SELECT ReportDate,
--        FormatGroup,
--        Count(DISTINCT bibnum) as ItemCount,
--        Sum(ei.itemcount) as InventoryCount
-- FROM   ElementInventory ei
--        INNER JOIN ItemCollection ic
--                ON ei.ItemCollectionCode = ic.Code
-- GROUP  BY ReportDate,
--           FormatGroup 